In [1]:
import os
import math
import numpy as np
import string

import openrtdynamics2.lang as dy
import openrtdynamics2.py_execute as dyexe
import openrtdynamics2.code_generation_helper as cgh
from openrtdynamics2.targets import TargetTemplate
from openrtdynamics2.manifest_import import get_all_inputs, get_all_outputs

from textwrap import indent


# An example for generating code in form of a Simulink S-function

In [45]:
class CppSimulinkSFunction(TargetTemplate):
    """
        generate a simple main.cpp programm
    """

    def __init__(self, enable_tracing=False, sfun_name : str = 'sfunction'):
        TargetTemplate.__init__(self, enable_tracing)
        self.sfun_name = sfun_name

    def code_gen(self):
        # build code of system        
        res = TargetTemplate.code_gen(self)
        
        # get the system manifest and c++ class name
        m = res['manifest']
        simulation_name = m['api_name']
        
        # headers
        headers = '#include <stdio.h>\n'
        
        # define structs for I/O
        io_variables_define = simulation_name + '::Inputs inputs;\n' + simulation_name + '::Outputs outputs;\n'

        
        #
        # create a list of all system inputs
        #
        
        inputs = get_all_inputs( m )
        
        number_of_inputs = len(inputs)

        get_s_inputs            = '' 
        fill_input_values       = ''
        define_input_port_sizes = ''
        
        for input_name, v in inputs.items():
            
            port_number_str = str( v['port_number'] )
            
            get_s_inputs            += 'InputRealPtrsType uPtrs' + port_number_str + ' = ssGetInputPortRealSignalPtrs(S,' + port_number_str + ');\n'
            fill_input_values       += 'inputs.' + input_name + ' = *uPtrs' + port_number_str + '[0];\n'
            define_input_port_sizes += 'ssSetInputPortWidth(S, '+ port_number_str +', 1);\n'

            
        #
        # create a list of the system inputs needed to compute the blocks outputs
        #
        
        inputs_for_block_outputs = get_all_inputs( 
            m, 
            return_inputs_to_calculate_outputs=True, 
            return_inputs_to_reset_states=False,
            return_inputs_to_update_states=False
        )
        
        get_s_inputs_for_model_outputs      = '' 
        fill_input_values_for_model_outputs = ''
        set_direct_feedthrough              = ''
        
        for input_name, v in inputs_for_block_outputs.items():
            
            port_number_str = str( v['port_number'] )
            
            get_s_inputs_for_model_outputs      += 'InputRealPtrsType uPtrs' + port_number_str + ' = ssGetInputPortRealSignalPtrs(S,' + port_number_str + ');\n'
            fill_input_values_for_model_outputs += 'inputs.' + input_name + ' = *uPtrs' + port_number_str + '[0];\n'
            set_direct_feedthrough              += 'ssSetInputPortDirectFeedThrough(S, ' + port_number_str + ', 1);\n'
       
        #
        # create a list of the system inputs needed to update the systems states
        #
        
        inputs_for_state_update = get_all_inputs( 
            m, 
            return_inputs_to_calculate_outputs=False, 
            return_inputs_to_reset_states=False,
            return_inputs_to_update_states=True
        )
        
        get_s_inputs_for_state_update      = '' 
        fill_input_values_for_state_update = ''
        
        for input_name, v in inputs_for_state_update.items():
            
            port_number_str = str( v['port_number'] )
            
            get_s_inputs_for_state_update      += 'InputRealPtrsType uPtrs' + port_number_str + ' = ssGetInputPortRealSignalPtrs(S,' + port_number_str + ');\n'
            fill_input_values_for_state_update += 'inputs.' + input_name + ' = *uPtrs' + port_number_str + '[0];\n'
       
            
        #
        # create a list of the system outputs
        #
        
        outputs = get_all_outputs( m )
        number_of_outputs = len(outputs)

        get_outputs              = ''
        fill_output_values       = ''
        define_output_port_sizes = ''
        
        j = 0 # output port index for simulink, starting at 0
        for output_name, v in outputs.items():

            port_number_str = str( j )
            
            get_outputs              += 'real_T  *y' + port_number_str + ' = ssGetOutputPortRealSignal(S, ' + port_number_str + ');\n'
            fill_output_values       += 'y' + port_number_str + '[0] = outputs.' + output_name + ';\n'
            define_output_port_sizes += 'ssSetOutputPortWidth(S, ' + port_number_str + ', 1);\n'
            
            j += 1
        
        
        
        
        
        
        
        # filename of cpp file
        cpp_fname = self.sfun_name + '.cpp'

        # code template
        main_fn = """

/*  File    : """ + cpp_fname + """
 *  Abstract:
 *
 *      Example of an C++ S-function which stores an C++ object in
 *      the pointers vector PWork.
 *
 *  Copyright 1990-2013 The MathWorks, Inc.
 */

#include <iostream>

""" + headers + '\n' + res['algorithm_sourcecode'] + """



#define S_FUNCTION_LEVEL 2
#define S_FUNCTION_NAME  """ + self.sfun_name + """

/*
 * Need to include simstruc.h for the definition of the SimStruct and
 * its associated macro definitions.
 */
#include "simstruc.h"

#define IS_PARAM_DOUBLE(pVal) (mxIsNumeric(pVal) && !mxIsLogical(pVal) &&\
!mxIsEmpty(pVal) && !mxIsSparse(pVal) && !mxIsComplex(pVal) && mxIsDouble(pVal))


/*====================*
 * S-function methods *
 *====================*/

#define MDL_CHECK_PARAMETERS
#if defined(MDL_CHECK_PARAMETERS)  && defined(MATLAB_MEX_FILE)
/*
 * Check to make sure that each parameter is 1-d and positive
 */
static void mdlCheckParameters(SimStruct *S)
{

    const mxArray *pVal0 = ssGetSFcnParam(S,0);

    if ( !IS_PARAM_DOUBLE(pVal0)) {
        ssSetErrorStatus(S, "Parameter to S-function must be a double scalar");
        return;
    } 
}
#endif


/* Function: mdlInitializeSizes ===============================================
 * Abstract:
 *    The sizes information is used by Simulink to determine the S-function
 *    block's characteristics (number of inputs, outputs, states, etc.).
 */
static void mdlInitializeSizes(SimStruct *S)
{
    ssSetNumSFcnParams(S, 1);  /* Number of expected parameters */
#if defined(MATLAB_MEX_FILE)
    if (ssGetNumSFcnParams(S) == ssGetSFcnParamsCount(S)) {
        mdlCheckParameters(S);
        if (ssGetErrorStatus(S) != NULL) {
            return;
        }
    } else {
        return; /* Parameter mismatch will be reported by Simulink */
    }
#endif
    ssSetSFcnParamTunable(S, 0, 0);

    // number of cont and discrete states
    ssSetNumContStates(S, 0);
    ssSetNumDiscStates(S, 0);

    // number of input ports
    if (!ssSetNumInputPorts(S, """ + str(number_of_inputs) + """  )) return;
    
    // ssSetInputPortWidth(S, 0, 1);
    // ssSetInputPortWidth(S, 1, 1);
""" + indent( define_input_port_sizes, '    ' ) + """

    // ssSetInputPortDirectFeedThrough(S, 1, 1);
    // ssSetInputPortDirectFeedThrough(S, 0, 1);

""" + indent( set_direct_feedthrough, '    ' ) + """





    // number of output ports
    if (!ssSetNumOutputPorts(S, """ + str(number_of_outputs) + """)) return;
    
    // ssSetOutputPortWidth(S, 0, 1);
    // ssSetOutputPortWidth(S, 1, 1);

""" + indent( define_output_port_sizes, '    ' ) + """



    // sample times
    ssSetNumSampleTimes(S, 1);
    
    // storage
    ssSetNumRWork(S, 0);
    ssSetNumIWork(S, 0);
    ssSetNumPWork(S, 1); // reserve element in the pointers vector
    ssSetNumModes(S, 0); // to store a C++ object
    ssSetNumNonsampledZCs(S, 0);

    // operating point
    ssSetOperatingPointCompliance(S, USE_DEFAULT_OPERATING_POINT);

    // general options
    ssSetOptions(S, 0);

    // ssSetOptions(S,
    //              SS_OPTION_WORKS_WITH_CODE_REUSE |
    //              SS_OPTION_EXCEPTION_FREE_CODE |
    //             SS_OPTION_USE_TLC_WITH_ACCELERATOR);
                 
}



/* Function: mdlInitializeSampleTimes =========================================
 * Abstract:
 *    This function is used to specify the sample time(s) for your
 *    S-function. You must register the same number of sample times as
 *    specified in ssSetNumSampleTimes.
 */
static void mdlInitializeSampleTimes(SimStruct *S)
{
    ssSetSampleTime(S, 0, mxGetScalar(ssGetSFcnParam(S, 0)));
    ssSetOffsetTime(S, 0, 0.0);
    ssSetModelReferenceSampleTimeDefaultInheritance(S);
}

#define MDL_START  /* Change to #undef to remove function */
#if defined(MDL_START) 
  /* Function: mdlStart =======================================================
   * Abstract:
   *    This function is called once at start of model execution. If you
   *    have states that should be initialized once, this is the place
   *    to do it.
   */
  static void mdlStart(SimStruct *S)
  {
      ssGetPWork(S)[0] = (void *) new """ + simulation_name + """; // store new C++ object in the
  }                                            // pointers vector
#endif /*  MDL_START */

/* Function: mdlOutputs =======================================================
 * Abstract:
 *    In this function, you compute the outputs of your S-function
 *    block.
 */
static void mdlOutputs(SimStruct *S, int_T tid)
{
    """ + simulation_name + """ *c = (""" + simulation_name + """ *) ssGetPWork(S)[0];
    
    // InputRealPtrsType uPtrs1 = ssGetInputPortRealSignalPtrs(S,0);
""" + indent(get_s_inputs_for_model_outputs, '    ') + """
    
    // outputs
    // real_T  *y1 = ssGetOutputPortRealSignal(S,0);
    // real_T  *y2 = ssGetOutputPortRealSignal(S,1);
""" + indent(get_outputs, '    ') + """

    // ORTD I/O structures
""" + indent( io_variables_define, '    ' ) + """
    
    
    // inputs.u1 = *uPtrs1[0];
    // inputs.u2 = 2;
""" + indent(fill_input_values_for_model_outputs, '    ') + """
    
    c->step( outputs, inputs, true, false, false ); 
    
    // y1[0] = outputs.y1;
    // y2[0] = outputs.y2;
""" + indent(fill_output_values, '    ') + """
    
    UNUSED_ARG(tid);                             // object
}                                                


#define MDL_UPDATE
/* Function: mdlUpdate ========================================================
 * Abstract:
 *    This function is called once for every major integration time step.
 *    Discrete states are typically updated here, but this function is useful
 *    for performing any tasks that should only take place once per integration
 *    step.
 */
static void mdlUpdate(SimStruct *S, int_T tid)
{
    InputRealPtrsType uPtrs  = ssGetInputPortRealSignalPtrs(S,0);
    """ + simulation_name + """ *c = (""" + simulation_name + """ *) ssGetPWork(S)[0];

""" + indent( io_variables_define, '    ' ) + """

    // InputRealPtrsType uPtrs1 = ssGetInputPortRealSignalPtrs(S,0);
""" + indent(get_s_inputs_for_state_update, '    ') + """


    UNUSED_ARG(tid); /* not used in single tasking mode */
    
    // inputs.u1 = *uPtrs1[0]; // U(i);
    // inputs.u2 = 2;
""" + indent(fill_input_values_for_state_update, '    ') + """


    c->step( outputs, inputs, false, true, false ); 
}












/* Function: mdlTerminate =====================================================
 * Abstract:
 *    In this function, you should perform any actions that are necessary
 *    at the termination of a simulation.  For example, if memory was
 *    allocated in mdlStart, this is the place to free it.
 */
static void mdlTerminate(SimStruct *S)
{
    """ + simulation_name + """ *c = (""" + simulation_name + """ *) ssGetPWork(S)[0]; // retrieve and destroy C++
    delete c;                                  // object in the termination
}                                              // function
/*======================================================*
 * See sfuntmpl.doc for the optional S-function methods *
 *======================================================*/

/*=============================*
 * Required S-function trailer *
 *=============================*/

#ifdef  MATLAB_MEX_FILE    /* Is this file being compiled as a MEX-file? */
#include "simulink.c"      /* MEX-file interface mechanism */
#else
#include "cg_sfun.h"       /* Code generation registration function */
#endif


        
        """
        
        #
        # build main.cpp
        #
        main_cpp = main_fn
        
        
        self.files[cpp_fname] = main_cpp
        
        # return
        return res

In [46]:
dy.clear()

system = dy.enter_system()

# define system inputs
u1               = dy.system_input( dy.DataTypeFloat64(1), name='u1', default_value=1.0,  value_range=[0, 25], title="input #1")
u2               = dy.system_input( dy.DataTypeFloat64(1), name='u2', default_value=2.0,  value_range=[0, 25], title="input #2")


y1 = dy.signal()                         # introduce variable y
x = y1 + u1                              # x[k] = y[k] + u[k]
y1 << dy.delay(x, initial_state = 2.0)   # y[k+1] = y[k] + x[k], y[0] = 2.0

# some second output
y2 = y1 + u2

# define output(s)
dy.append_output(y1, 'y1')
dy.append_output(y2, 'y2')

# generate code using the target defined above
code_gen_results = dy.generate_code(template=CppSimulinkSFunction(sfun_name='sfunction_demo'), folder='generated/simulink_sfunction')

compiling system simulation (level 0)... 
Generated code will be written to generated/simulink_sfunction .
writing file generated/simulink_sfunction/simulation_manifest.json
writing file generated/simulink_sfunction/sfunction_demo.cpp


In [47]:
dy.clear()

system = dy.enter_system()

# define system inputs
u1               = dy.system_input( dy.DataTypeFloat64(1), name='u1', default_value=1.0,  value_range=[0, 25], title="input #1")
u2               = dy.system_input( dy.DataTypeFloat64(1), name='u2', default_value=2.0,  value_range=[0, 25], title="input #2")


y1 = dy.signal()                         # introduce variable y
x = y1 + u1                              # x[k] = y[k] + u[k]
y1 << dy.delay(x, initial_state = 2.0)   # y[k+1] = y[k] + x[k], y[0] = 2.0

# some second output
y2 = y1 + dy.dtf_lowpass_1_order(u2, 0.9)   # dy.rate_limit( u2, 0.1, -1, 1 )

# define output(s)
dy.append_output(y1, 'y1')
dy.append_output(y2, 'y2')

# generate code using the target defined above
code_gen_results = dy.generate_code(template=CppSimulinkSFunction(sfun_name='sfunction_demo'), folder='generated/simulink_sfunction')

compiling system simulation (level 0)... 
Generated code will be written to generated/simulink_sfunction .
writing file generated/simulink_sfunction/simulation_manifest.json
writing file generated/simulink_sfunction/sfunction_demo.cpp


### Generated manifest and source code 

In [4]:
# show the manifest

# from IPython.display import JSON, Code
# JSON(code_gen_results['manifest'], expanded=True)

In [5]:
# show the generated source code

Code(data=code_gen_results['files']['main.cpp'], language='c++')

NameError: name 'Code' is not defined